In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
# ! git pull

## Add import path

In [ ]:
import gc
import os
import sys

In [ ]:
def add_library_level(level=4):
    suf_path = ['..']
    path = '..'
    for i in range(0, level):
        join_path = suf_path * i
        path = '/'.join(join_path)
        module_path = os.path.abspath(os.path.join(path))
        if module_path not in sys.path:
            sys.path.append(module_path)
            print(f'Appendeding {path}')

In [ ]:
add_library_level(level=5)

## Organize imports

In [ ]:
import multiprocessing

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.fca_utils import *

In [ ]:
from src.lattmc.tc.transcoder_analyzers import ConceptAnalysis, init_analyzer

In [ ]:
from src.lattmc.sae.nlp_sae_utils import init_device, gen_concept

In [ ]:
import logging

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2025

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
device = init_device()
device

In [ ]:
torch.__version__

In [ ]:
np.__version__

In [ ]:
# np.set_printoptions(precision=4, suppress=True)

## Initialize Paths

In [ ]:
PATH = Path('data')
GPT2 = PATH / 'transcoders' / 'gpt2'
OWT_TOKENS_DIR = GPT2 / 'owt_tokens'
TOKENS_PATH = OWT_TOKENS_DIR / 'owt_tokens_torch.pt'
OWT_TOKENS_DIR.mkdir(exist_ok=True, parents=True)

## Load trancoders

In [ ]:
layers = list(range(12))
layers = [0, 4, 6, 8, 10, 11]

In [ ]:
tr_analyzer = init_analyzer(
    layers,
    TOKENS_PATH,
    GPT2,
    device=device
)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Detect sences

In [ ]:
concept_an = ConceptAnalysis(' The important sense is a sense of smell, odour, sense is also vision and hearing', tr_analyzer)

In [ ]:
concept_an.analyze_concepts()

In [ ]:
t_idcs = [8, 16]
layer = 0

In [ ]:
concept_an.gen_text(t_idcs, layer)

In [ ]:
layer = 8

In [ ]:
concept_an.gen_text(t_idcs, layer)

In [ ]:
layer = 11

In [ ]:
concept_an.gen_text(t_idcs, layer)

## Experiments Pos and Negs Black

In [ ]:
tok_indx = 143

In [ ]:
text_detoken = tr_analyzer.to_clean(tr_analyzer.tokens[tok_indx])
text_detoken

In [ ]:
concept_an = ConceptAnalysis(text_detoken, tr_analyzer)

In [ ]:
concept_an.analyze_concepts()

In [ ]:
i, j = 27, 28
t_idcs = [28, 127]

In [ ]:
layer_0 = 0

In [ ]:
concept_an.tr_utils.transcoder.background_dets = None

In [ ]:
concept_an.gen_text(t_idcs, layer_0)

In [ ]:
det_22 = concept_an.detected_vs[layer_0][22]
det_22

In [ ]:
v_b1 = det_22[28][32]
v_b2 = det_22[127][32]

In [ ]:
v_b3 = det_22[127][96]
v_b4 = det_22[127][119]

In [ ]:
v_b = join(v_b1, v_b2)

In [ ]:
topK(v_b1, 20)

In [ ]:
topK(v_b3, 20)

In [ ]:
topK(v_b4, 20)

In [ ]:
np.all(v_b1 == v_b2)

In [ ]:
layer_8 = 8

In [ ]:
concept_an.gen_text(t_idcs, layer_8)

In [ ]:
concept_an.detected_vs[layer_8]

In [ ]:
layer_11 = 11

In [ ]:
concept_an.gen_text(t_idcs, layer_11)

In [ ]:
concept_an.detected_vs[layer_11][3]

In [ ]:
concept_an.detected_vs[layer_11][8]

In [ ]:
v_white = concept_an.detected_vs[layer_11][8][28][62]
v_black = concept_an.detected_vs[layer_11][8][127][34]

In [ ]:
topK(v_white, 10)

In [ ]:
topK(v_black, 10)

In [ ]:
v_meet = meet(v_white, v_black)

In [ ]:
vals, indcs = topK(v_meet, 10)
vals, indcs

In [ ]:
c_meet = concept_an.gen_concept(indcs[1], vals[1], layer_11)
c_meet

In [ ]:
concept_an.gen_and_print(indcs[1:4], vals[1:4], layer_11, with_text=True, limit=20)

In [ ]:
concept_an.gen_and_print([21836], [17], layer_11, with_text=True, limit=20)